In [167]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import datetime

%config IPCompleter.greedy=True 
%matplotlib inline  

import os

In [168]:
class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

# For long  and Short

#input

root_path=r'D:\DataSets\Build-DataSet\S50F15M_LS\Label&Feature 3Q20To1Q21\Long_Label_3q20-1q21'

marklabelPath=os.path.abspath(root_path)
feature_file=os.path.abspath(f'{root_path}\\Feature_Long_15M_3q20-1q21.csv')

train_file=os.path.abspath(f'{root_path}\\MLTrain-Long_S50M15_3Q20To1Q21.csv')
ab_file=f'{root_path}\\ABTrain-Long_S50M15_3Q20To1Q21.csv'


combine_path=r'D:\DataSets\Build-DataSet\S50F15M_LS\Label&Feature 3Q20To1Q21'
existing_train_file=f'{combine_path}\\ML-Long_S50M15_07To2Q20-Train_Norm.csv'


new_traing_file=f'{combine_path}\\ML-Long_S50M15_07To1Q21.csv'

labelName="LongSignal"           
#labelName="ShortSignal"  



In [170]:
# For  Short

#input

root_path=r'D:\DataSets\Build-DataSet\S50F15M_LS\Label&Feature 3Q20To1Q21\Short_Label_3q20-1q21'

marklabelPath=os.path.abspath(root_path)
feature_file=os.path.abspath(f'{root_path}\\Feature_Short_15M_3q20-1q21.csv')

train_file=os.path.abspath(f'{root_path}\\MLTrain-Short_S50M15_3Q20To1Q21.csv')
ab_file=f'{root_path}\\ABTrain-Short_S50M15_3Q20To1Q21.csv'


combine_path=r'D:\DataSets\Build-DataSet\S50F15M_LS\Label&Feature 3Q20To1Q21'
existing_train_file=f'{combine_path}\\ML-Short_S50M15_07To2Q20-Train_Norm.csv'


new_traing_file=f'{combine_path}\\ML-Short_S50M15_07To1Q21.csv'

     
labelName="ShortSignal" 

In [171]:
print(marklabelPath," = ",os.path.exists(marklabelPath))
print(feature_file," = ",os.path.exists (feature_file))

D:\DataSets\Build-DataSet\S50F15M_LS\Label&Feature 3Q20To1Q21\Short_Label_3q20-1q21  =  True
D:\DataSets\Build-DataSet\S50F15M_LS\Label&Feature 3Q20To1Q21\Short_Label_3q20-1q21\Feature_Short_15M_3q20-1q21.csv  =  True


# Build Train Feature File

For binary classification isn't nessary to initialize default label

# Create TrainData by merge SignalLable to Feature 

In [172]:
def CleanFeatureData(df):
    df["datetime"]=pd.to_datetime(df["Date/Time"],format='%d/%m/%Y %H:%M:%S',dayfirst=True)
    df.drop(columns=['Ticker','Date/Time'],inplace=True)
    df.set_index('datetime',inplace=True)
    df.dropna(inplace=True)

In [173]:
def BuildLabelData(sourceFile,datetime_format):
 df=pd.read_csv(sourceFile)
 
 df["datetime"]=pd.to_datetime(df["datetime"],format=datetime_format,dayfirst=True)
 df.set_index('datetime',inplace=True)
 df.drop(columns=['open','high','low','close'],inplace=True)
 df.rename(columns={'signal':labelName},inplace=True)    

 print(df.shape)
 print(df[labelName].value_counts())
 
 return df

In [174]:
def IterateMarkLabel(temp_df_feature,markFile,timeFormat):

    
    df_label=BuildLabelData(markFile,timeFormat)
    '''
    print("df_label=", df_label.shape)
    print(df_label.head(3))
    print("==============================================")
    '''
    
    df_train=temp_df_feature.merge(df_label,how='left',on='datetime')
    '''
    print("df_trainl=", df_train.shape)
    print(df_train.head(3))
    print("##################################################")
    '''
    
    return df_train
    

# Load Features File(Explore from Amibroker)

In [175]:
df_indy=pd.read_csv(feature_file)
CleanFeatureData(df_indy)
print(df_indy.info())
df_indy.tail(10)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4005 entries, 2020-07-01 09:45:00 to 2021-03-31 16:45:00
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   open                    4005 non-null   float64
 1   high                    4005 non-null   float64
 2   low                     4005 non-null   float64
 3   close                   4005 non-null   float64
 4   indy_ma-220             4005 non-null   float64
 5   indy_ma-550             4005 non-null   float64
 6   indy_hh-220             4005 non-null   float64
 7   indy_ll-220             4005 non-null   float64
 8   indy_mid-220            4005 non-null   float64
 9   indy_hh2-550            4005 non-null   float64
 10  indy_ll2-550            4005 non-null   float64
 11  indy_mid2-550           4005 non-null   float64
 12  indy_macd110-440        4005 non-null   float64
 13  indy_signal110-440-110  4005 non-null   float64
 14  indy

,open,high,low,close,indy_ma-220,indy_ma-550,indy_hh-220,indy_ll-220,indy_mid-220,indy_hh2-550,...,indy_mid2-550,indy_macd110-440,indy_signal110-440-110,indy_hist_macd110-440,indy_rsi25-ma20,indy_6ATRTrail_DC-110,cate_3trend-220_ma110,cate_2trend-550_ma110,cate_rannkHL550-ma66,cate_CombineTrend
datetime,,,,,,,,,,,,,,,,,,,,,
2021-03-31 14:30:00,967.2,968.4,966.6,968.2,0.99,0.99,1.01,0.98,0.001,1.02,...,0.001,2.07,1.33,0.75,50.06,0.99,2,1,3,4
2021-03-31 14:45:00,968.3,968.3,967.5,967.6,1.00,0.99,1.01,0.98,0.001,1.02,...,0.001,2.16,1.29,0.87,49.51,0.99,2,1,3,4
2021-03-31 15:00:00,967.7,969.2,966.2,966.8,1.00,0.99,1.01,0.98,0.001,1.02,...,0.001,2.22,1.26,0.96,49.21,0.99,2,1,3,4
2021-03-31 15:15:00,966.7,967.2,966.6,966.7,1.00,0.99,1.01,0.98,0.001,1.02,...,0.001,2.28,1.23,1.05,48.76,0.99,2,1,3,4
2021-03-31 15:30:00,966.6,967.0,965.0,965.4,1.00,0.99,1.01,0.98,0.001,1.02,...,0.001,2.33,1.21,1.13,48.30,1.00,2,1,3,4
2021-03-31 15:45:00,965.3,966.4,964.6,965.3,1.00,0.99,1.01,0.98,0.001,1.02,...,0.001,2.39,1.18,1.20,47.74,1.00,2,1,3,4
2021-03-31 16:00:00,965.2,966.4,965.0,965.6,1.00,0.99,1.01,0.98,0.001,1.02,...,0.001,2.45,1.16,1.29,47.00,1.00,2,1,3,4
2021-03-31 16:15:00,965.5,966.9,965.1,965.4,1.00,0.99,1.01,0.98,0.001,1.02,...,0.001,2.53,1.14,1.39,46.28,1.00,2,1,3,4
2021-03-31 16:30:00,965.6,965.9,965.1,965.8,1.00,0.99,1.01,0.98,0.001,1.02,...,0.001,2.58,1.12,1.46,46.08,1.00,2,1,3,4


# mark file

#'%d/%m/%Y %H:%M:%S'

# update label in mark file

#'%Y-%m-%d %H:%M:%S'

In [176]:
#For long / short
# 2011 mark end oct11 and start 2012 at nov12
#'%d/%m/%Y %H:%M:%S'
#'%Y-%m-%d %H:%M:%S'
xact='Short'
markFileList=[

    [ df_indy.loc['07-2020':'03-2021',:],f'{marklabelPath}\\Label_{xact}_3q20-1q21.csv','%d/%m/%Y %H:%M:%S']
    
#     ,[ df_indy.loc['01-2009':'10-2011'],f'{marklabelPath}\\S50_09-11_LongOnly.csv','%d/%m/%Y %H:%M:%S']
    
#     ,[ df_indy.loc['11-2011':'12-2013'],f'{marklabelPath}\\S50_12-13_LongOnly.csv','%d/%m/%Y %H:%M:%S']
    
#     ,[ df_indy.loc['01-2014':'12-2015'],f'{marklabelPath}\\S50_14-15_LongOnly.csv','%d/%m/%Y %H:%M:%S']
     


]

print("no.markFile",len(markFileList))

no.markFile 1


In [159]:

listDFTrain=[]
for  mark in  markFileList:
     df_subFeature=mark[0]
     markSignalPath=mark[1]
     timeformat=mark[2]
     print(markSignalPath)   
     xdf_train=IterateMarkLabel(df_subFeature,markSignalPath,timeformat)
     listDFTrain.append(xdf_train)



D:\DataSets\Build-DataSet\S50F15M_LS\Label&Feature 3Q20To1Q21\Long_Label_3q20-1q21\Label_Long_3q20-1q21.csv
(78113, 1)
0    76806
1     1307
Name: LongSignal, dtype: int64


In [160]:
trainDF= pd.concat(listDFTrain)
print(trainDF.shape)
display('trainDF.head(5)','trainDF.tail(5)')


(4005, 22)


,open,high,low,close,indy_ma-550,indy_ma-1100,indy_hh-550,indy_ll-550,indy_mid-550,indy_hh2-1100,...,indy_macd110-440,indy_signal110-440-110,indy_hist_macd110-440,indy_rsi25-ma20,indy_6ATRTrail_DC-110,cate_3trend-550_ma110,cate_2trend-1100_ma220,cate_rannkHL1100-ma66,cate_CombineTrend,LongSignal
datetime,,,,,,,,,,,,,,,,,,,,,
2020-07-01 09:45:00,880.0,882.2,878.3,881.0,1.03,1.0,1.11,0.97,0.0012,1.11,...,-33.86,-23.94,-9.92,56.26,1.0,2,1,4,3,0
2020-07-01 10:00:00,881.2,881.5,879.4,880.4,1.03,1.0,1.11,0.97,0.0012,1.11,...,-33.99,-24.14,-9.85,55.89,1.0,2,1,4,3,0
2020-07-01 10:15:00,880.3,882.5,880.2,881.3,1.03,1.0,1.11,0.97,0.0012,1.11,...,-34.11,-24.34,-9.76,55.53,1.0,2,1,4,3,0
2020-07-01 10:30:00,881.2,883.7,880.2,883.7,1.03,1.0,1.11,0.97,0.0012,1.11,...,-34.20,-24.54,-9.65,55.22,1.0,2,1,4,3,0
2020-07-01 10:45:00,883.6,884.2,882.2,882.9,1.03,1.0,1.11,0.97,0.0012,1.11,...,-34.29,-24.75,-9.54,54.94,1.0,2,1,4,3,0
,open,high,low,close,indy_ma-550,indy_ma-1100,indy_hh-550,indy_ll-550,indy_mid-550,indy_hh2-1100,...,indy_macd110-440,indy_signal110-440-110,indy_hist_macd110-440,indy_rsi25-ma20,indy_6ATRTrail_DC-110,cate_3trend-550_ma110,cate_2trend-1100_ma220,cate_rannkHL1100-ma66,cate_CombineTrend,LongSignal
datetime,,,,,,,,,,,,,,,,,,,,,
2021-03-31 15:45:00,965.3,966.4,964.6,965.3,0.99,0.98,1.02,0.95,0.001,1.02,...,2.39,1.18,1.20,47.74,1.0,3,0,3,4,0
2021-03-31 16:00:00,965.2,966.4,965.0,965.6,0.99,0.98,1.02,0.95,0.001,1.02,...,2.45,1.16,1.29,47.00,1.0,3,0,3,4,0


# Export File to TrainML and Amibroker

In [161]:
trainDF.to_csv(train_file)

In [162]:
aux=labelName
df_amibroker=trainDF[['open','high','low','close',aux]]
df_amibroker.head(5)
df_amibroker.to_csv(ab_file)   

# Analyze File TrainLabel

In [163]:
print(trainDF[labelName].value_counts())
print("###############################")
print(round(trainDF[labelName].value_counts(normalize=True)*100,0))

0    2698
1    1307
Name: LongSignal, dtype: int64
###############################
0    67.0
1    33.0
Name: LongSignal, dtype: float64


# MergeTrain_NewToExisting

In [164]:


trainDF_Existing=pd.read_csv(existing_train_file,index_col='datetime',parse_dates =['datetime'],dayfirst=True)

# for test diff
# trainDF_Existing.rename(columns={'indy_ma-550':'indy_MA-550','LongSignal':'XSignal'},inplace=True)

print(trainDF_Existing.shape)
print(trainDF_Existing.info())


display('trainDF_Existing.head(5)','trainDF_Existing.tail(5)')




(71001, 22)
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 71001 entries, 2007-01-03 09:45:00 to 2020-06-30 16:45:00
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   open                    71001 non-null  float64
 1   high                    71001 non-null  float64
 2   low                     71001 non-null  float64
 3   close                   71001 non-null  float64
 4   indy_ma-550             71001 non-null  float64
 5   indy_ma-1100            71001 non-null  float64
 6   indy_hh-550             71001 non-null  float64
 7   indy_ll-550             71001 non-null  float64
 8   indy_mid-550            71001 non-null  float64
 9   indy_hh2-1100           71001 non-null  float64
 10  indy_ll2-1100           71001 non-null  float64
 11  indy_mid2-1100          71001 non-null  float64
 12  indy_macd110-440        71001 non-null  float64
 13  indy_signal110-440-110  71001 non-null  floa

,open,high,low,close,indy_ma-550,indy_ma-1100,indy_hh-550,indy_ll-550,indy_mid-550,indy_hh2-1100,...,indy_macd110-440,indy_signal110-440-110,indy_hist_macd110-440,indy_rsi25-ma20,indy_6ATRTrail_DC-110,cate_3trend-550_ma110,cate_2trend-1100_ma220,cate_rannkHL1100-ma66,cate_CombineTrend,LongSignal
datetime,,,,,,,,,,,,,,,,,,,,,
2007-01-03 09:45:00,460.0,460.5,458.0,458.0,1.10,1.11,1.15,0.91,0.0022,1.15,...,-22.50,-27.20,4.70,39.11,1.03,1,0,3,2,0
2007-01-03 10:00:00,458.0,464.5,457.6,463.9,1.09,1.10,1.14,0.89,0.0022,1.14,...,-22.50,-27.19,4.68,39.05,1.02,1,0,3,2,0
2007-01-03 10:15:00,463.9,464.0,459.2,460.0,1.09,1.11,1.15,0.90,0.0022,1.15,...,-22.53,-27.17,4.64,38.91,1.03,1,0,3,2,0
2007-01-03 10:30:00,460.0,461.0,459.8,460.0,1.09,1.11,1.15,0.90,0.0022,1.15,...,-22.57,-27.15,4.58,38.86,1.03,1,0,3,2,0
2007-01-03 10:45:00,460.0,460.7,460.0,460.5,1.09,1.10,1.14,0.90,0.0022,1.15,...,-22.60,-27.13,4.53,38.65,1.03,1,0,3,2,0
,open,high,low,close,indy_ma-550,indy_ma-1100,indy_hh-550,indy_ll-550,indy_mid-550,indy_hh2-1100,...,indy_macd110-440,indy_signal110-440-110,indy_hist_macd110-440,indy_rsi25-ma20,indy_6ATRTrail_DC-110,cate_3trend-550_ma110,cate_2trend-1100_ma220,cate_rannkHL1100-ma66,cate_CombineTrend,LongSignal
datetime,,,,,,,,,,,,,,,,,,,,,
2020-06-30 15:45:00,883.2,883.8,881.7,882.7,1.03,1.0,1.11,0.97,0.0012,1.11,...,-33.34,-22.94,-10.41,57.53,1.00,2,1,4,3,0
2020-06-30 16:00:00,882.5,883.1,880.2,882.0,1.03,1.0,1.11,0.97,0.0012,1.11,...,-33.44,-23.14,-10.30,57.88,1.00,2,1,4,3,0


# check all column mush be matched both position and name

In [165]:
newVSold_FeatureCols=list(zip(trainDF.columns.tolist(),trainDF_Existing.columns.tolist()))

diff_FeatureCols=[x for x in newVSold_FeatureCols  if x[0].strip()!= x[1].strip()]

print(newVSold_FeatureCols)
print("---------------------------------------------------------------------")
print(diff_FeatureCols)

[('open', 'open'), ('high', 'high'), ('low', 'low'), ('close', 'close'), ('indy_ma-550', 'indy_ma-550'), ('indy_ma-1100', 'indy_ma-1100'), ('indy_hh-550', 'indy_hh-550'), ('indy_ll-550', 'indy_ll-550'), ('indy_mid-550', 'indy_mid-550'), ('indy_hh2-1100', 'indy_hh2-1100'), ('indy_ll2-1100', 'indy_ll2-1100'), ('indy_mid2-1100', 'indy_mid2-1100'), ('indy_macd110-440', 'indy_macd110-440'), ('indy_signal110-440-110', 'indy_signal110-440-110'), ('indy_hist_macd110-440', 'indy_hist_macd110-440'), ('indy_rsi25-ma20', 'indy_rsi25-ma20'), ('indy_6ATRTrail_DC-110', 'indy_6ATRTrail_DC-110'), ('cate_3trend-550_ma110', 'cate_3trend-550_ma110'), ('cate_2trend-1100_ma220', 'cate_2trend-1100_ma220'), ('cate_rannkHL1100-ma66', 'cate_rannkHL1100-ma66'), ('cate_CombineTrend', 'cate_CombineTrend'), ('LongSignal', 'LongSignal')]
---------------------------------------------------------------------
[]


In [166]:
if len(diff_FeatureCols)==0 :                                      
    dfx=pd.concat([trainDF_Existing,trainDF],axis=0)
    dup_df=dfx[dfx.index.duplicated()]
    if len(dup_df)==0 :
     dfx.to_csv(new_traing_file)
    else:
     print("Found duplicated  datetime index")    
     print(dup_df)          
                                       
else:
    print("Some columns didn't match both  name and position")                                   
    print(diff_FeatureCols)                                  